In [1]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
sys.path.insert(0, os.path.abspath('../'))
from src.lib import utils
from src.lib import features as ft
from src.lib import centroids as centutils

In [2]:
df_first_path = '../results/first_features.csv'
df_negsi_path = '../results/NegsiRNA_features.csv'
df_test_path = '../results/Test_features.csv'

df_first = pd.read_csv(df_first_path, index_col=0)
df_negsi = pd.read_csv(df_negsi_path, index_col=0)
df_test = pd.read_csv(df_test_path, index_col=0)

df_inter_first_negsi = df_first.merge(df_negsi, left_index=True, right_index=True)
df_inter_first_test = df_first.merge(df_test, left_index=True, right_index=True)
df_inter_negsi_test = df_negsi.merge(df_test, left_index=True, right_index=True)

df_test_no_intersection = df_test.drop(df_inter_first_test.index)

df_first['folder'] = 'first'
df_negsi['folder'] = 'NegsiRNA'
df_test_no_intersection['folder'] = 'Test'

df_all = pd.concat([df_first, df_negsi, df_test_no_intersection])
df_all.fillna(0)

,cell_area_ratio,white_area_ratio,e2e_n,e2e_distance_mean,e2e_distance_std,e2e_eu_distance_mean,e2e_eu_distance_std,e2e_distance_ratio_mean,e2e_distance_ratio_std,j2e_n,...,nodes_std,medial_axis_mean,medial_axis_std,voronoi_approx_mean,voronoi_approx_std,std_filter_mean,std_filter_std,entropy_filter_mean,entropy_filter_std,folder
ACTB_B2_CA,0.603739,0.280296,8,11.673097,16.213742,10.770630,13.984421,0.970518,0.047607,361,...,0.344300,9.385630,6.708586,9.997991,6.821478,8.198705,5.645586,4.122258,0.566555,first
ACTN4_L3_NO,0.643184,0.264147,2,5.328427,0.707107,4.621320,0.535534,0.868272,0.014719,508,...,0.329785,7.979354,5.130908,8.927850,5.022501,5.150305,3.239704,3.679391,0.585222,first
ACTB_B2_NS,0.529037,0.393422,2,6.621320,4.535534,5.914214,4.363961,0.872069,0.061719,348,...,0.340088,13.342543,8.466754,14.887825,8.152079,8.104494,5.593832,4.099480,0.562513,first
NegsiRNA_B1_NO,0.584348,0.361675,3,2.276142,1.863171,2.179106,1.696244,0.978017,0.038075,414,...,0.359689,10.740390,6.388646,11.921827,6.126879,5.706624,3.521712,3.810957,0.542083,first
NegsiRNA_B1_CA,0.672008,0.249850,5,25.802439,24.656564,23.709567,23.061234,0.897672,0.111001,291,...,0.372899,9.099332,6.238017,9.655176,6.140472,5.103108,3.393098,3.680185,0.573167,first
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PALLD_F2_CA,0.532583,0.352931,1,13.000000,0.000000,13.000000,0.000000,1.000000,0.000000,272,...,0.333959,10.779967,6.281335,11.602166,6.057167,7.477804,5.024334,4.046653,0.558139,Test
EZR_C3_NO,0.648874,0.290315,1,2.000000,0.000000,2.000000,0.000000,1.000000,0.000000,312,...,0.344878,8.982927,5.137285,9.530259,4.744985,5.470714,3.365124,3.763924,0.556228,Test
DNMBP_A3_NO,0.603660,0.226212,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,411,...,0.350006,7.369823,4.892748,7.902959,4.893513,4.808559,2.676384,3.642820,0.560697,Test
NUMB_E3_CA,0.654215,0.271097,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,275,...,0.347745,8.482209,5.014825,9.041384,4.877564,4.326597,2.793765,3.543888,0.565170,Test


In [29]:
def get_histogram_values(img, prefix):
    d = {}
    non_zero = img[np.where(img != 0)]
    mean, std = non_zero.mean(), non_zero.std()
    hist, bins = np.histogram(non_zero, bins=20)
    d[f'{prefix}_mean'] = mean
    d[f'{prefix}_std'] = std
    d.update({f'{prefix}_value_{index}-{index + 1}': hist[index] for index in range(len(hist))})
    d.update({f'{prefix}_bin_{index}': bins[index] for index in range(len(bins))})
    return d

In [35]:
template_path = '../data/{folder}/{img_type}/{name}{suffix}.tif'
L_8 = []
L_16 = []
for img_name, row in df_all.iterrows():
    print(img_name)
    d_8 = {'img_name': img_name, 'folder': row['folder']}
    d_16 = d_8.copy()
    
    nuclei_img, original_img, seg_img, sk_img = utils.load_images(os.path.join('../data/', row['folder']), img_name)
    
    std_img_8 = ft.texture_std_filter(utils.convert_16_to_8(original_img), seg_img)
    std_img_16 = ft.texture_std_filter(original_img, seg_img)
    entropy_img_8 = ft.texture_entropy_filter(utils.convert_16_to_8(original_img), seg_img)

    d_std_8 = get_histogram_values(std_img_8, 'std_filter')
    d_std_16 = get_histogram_values(std_img_16, 'std_filter')
    d_entropy_8 = get_histogram_values(entropy_img_8, 'entropy_filter')

    d_8.update(d_std_8)
    d_8.update(d_entropy_8)

    d_16.update(d_std_16)
    d_16.update(d_entropy_8)

    L_8.append(d_8)
    L_16.append(d_16)


ACTB_B2_CA
ACTN4_L3_NO
ACTB_B2_NS
NegsiRNA_B1_NO
NegsiRNA_B1_CA
AKAP5_B2_CA
NegsiRNA_A2_NS
NegsiRNA_A2b_NS
AKT1_B1_CA
ADAM9_C2_NO
NegsiRNA_A2b_NO
ADD3_A2_CA
NegsiRNA_A2b_CA
ACTB_B2_NO
AKT1_B2_CA
NegsiRNA_A3_NS
NegsiRNA_A3_NO
NegsiRNA_A3_CA
ACTR3_C3_NO
ACTN4_L2_NO
NegsiRNA_C3c_CA
NegsiRNA_C3b_CA
NegsiRNA_E3_NO
NegsiRNA_L2_NS
NegsiRNA_L1_CA
NegsiRNA_H1_CA
NegsiRNA_H3_NO
NegsiRNA_A2_CA
NegsiRNA_G3_NS
NegsiRNA_B1_NO
NegsiRNA_I1_CA
NegsiRNA_F3_NS
NegsiRNA_C2_NO
NegsiRNA_B1_CA
NegsiRNA_D1_NS
NegsiRNA_H3_CA
NegsiRNA_G1_NS
NegsiRNA_C2_NS
NegsiRNA_F2_NS
NegsiRNA_I1_NO
NegsiRNA_A2_NO
NegsiRNA_A2_NS
NegsiRNA_B2_NO
NegsiRNA_A2b_NS
NegsiRNA_E3_NS
NegsiRNA_H2_NS
NegsiRNA_E1_NS
NegsiRNA_D3_NS
NegsiRNA_L1_NO
NegsiRNA_B3_NO
NegsiRNA_D2_CA
NegsiRNA_F2_NO
NegsiRNA_F1_NO
NegsiRNA_I3_NS
NegsiRNA_G1_NO
NegsiRNA_A2b_NO
NegsiRNA_A1b_CA
NegsiRNA_C1_NO
NegsiRNA_C1_CA
NegsiRNA_C3_CA
NegsiRNA_B3_NS
NegsiRNA_B1_NS
NegsiRNA_E2_NS
NegsiRNA_C3c_NO
NegsiRNA_I2_NS
NegsiRNA_C3b_NO
NegsiRNA_A1b_NO
NegsiRNA_I1_NS
NegsiRNA

In [38]:
df_8 = pd.DataFrame(L_8)
df_8

,img_name,folder,std_filter_mean,std_filter_std,std_filter_value_0-1,std_filter_value_1-2,std_filter_value_2-3,std_filter_value_3-4,std_filter_value_4-5,std_filter_value_5-6,...,entropy_filter_bin_11,entropy_filter_bin_12,entropy_filter_bin_13,entropy_filter_bin_14,entropy_filter_bin_15,entropy_filter_bin_16,entropy_filter_bin_17,entropy_filter_bin_18,entropy_filter_bin_19,entropy_filter_bin_20
0,ACTB_B2_CA,first,8.198705,5.645586,139182,206790,128928,75231,45199,26642,...,3.926577,4.109612,4.292647,4.475682,4.658718,4.841753,5.024788,5.207823,5.390858,5.573894
1,ACTN4_L3_NO,first,5.150305,3.239704,89054,214720,137704,80045,45294,24641,...,3.480613,3.693347,3.906082,4.118816,4.331551,4.544285,4.757019,4.969754,5.182488,5.395222
2,ACTB_B2_NS,first,8.104494,5.593832,109428,300743,200379,119752,72973,45828,...,3.599099,3.813986,4.028872,4.243758,4.458645,4.673531,4.888418,5.103304,5.318191,5.533077
3,NegsiRNA_B1_NO,first,5.706624,3.521712,153687,312835,192414,96681,46935,22788,...,3.693350,3.893229,4.093108,4.292987,4.492866,4.692745,4.892624,5.092503,5.292382,5.492261
4,NegsiRNA_B1_CA,first,5.103108,3.393098,163964,239714,107379,40823,18114,9013,...,3.400675,3.631361,3.862048,4.092735,4.323421,4.554108,4.784795,5.015482,5.246168,5.476855
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,PALLD_F2_CA,Test,7.477804,5.024334,179557,303509,169057,84565,43796,23866,...,3.726484,3.931751,4.137019,4.342287,4.547555,4.752822,4.958090,5.163358,5.368626,5.573894
241,EZR_C3_NO,Test,5.470714,3.365124,99396,243973,155410,86221,46162,24395,...,3.552562,3.768084,3.983606,4.199128,4.414650,4.630172,4.845695,5.061217,5.276739,5.492261
242,DNMBP_A3_NO,Test,4.808559,2.676384,65201,217528,143069,64145,25143,9988,...,3.147823,3.403781,3.659739,3.915697,4.171655,4.427613,4.683571,4.939529,5.195487,5.451445
243,NUMB_E3_CA,Test,4.326597,2.793765,265186,241531,84470,28812,10570,4256,...,3.614475,3.818583,4.022690,4.226798,4.430906,4.635014,4.839121,5.043229,5.247337,5.451445


In [39]:
df_16 = pd.DataFrame(L_16)
df_16

,img_name,folder,std_filter_mean,std_filter_std,std_filter_value_0-1,std_filter_value_1-2,std_filter_value_2-3,std_filter_value_3-4,std_filter_value_4-5,std_filter_value_5-6,...,entropy_filter_bin_11,entropy_filter_bin_12,entropy_filter_bin_13,entropy_filter_bin_14,entropy_filter_bin_15,entropy_filter_bin_16,entropy_filter_bin_17,entropy_filter_bin_18,entropy_filter_bin_19,entropy_filter_bin_20
0,ACTB_B2_CA,first,2097.093202,1445.999241,137679,207230,129259,75528,45298,26831,...,3.926577,4.109612,4.292647,4.475682,4.658718,4.841753,5.024788,5.207823,5.390858,5.573894
1,ACTN4_L3_NO,first,1315.729588,830.540230,93301,213085,136529,79385,44939,24461,...,3.480613,3.693347,3.906082,4.118816,4.331551,4.544285,4.757019,4.969754,5.182488,5.395222
2,ACTB_B2_NS,first,2073.007747,1432.761788,111255,299926,199723,119525,72912,45757,...,3.599099,3.813986,4.028872,4.243758,4.458645,4.673531,4.888418,5.103304,5.318191,5.533077
3,NegsiRNA_B1_NO,first,1458.410910,902.466772,148573,313583,194284,97967,47571,23106,...,3.693350,3.893229,4.093108,4.292987,4.492866,4.692745,4.892624,5.092503,5.292382,5.492261
4,NegsiRNA_B1_CA,first,1303.575069,869.809490,150426,246084,111954,42437,18659,9256,...,3.400675,3.631361,3.862048,4.092735,4.323421,4.554108,4.784795,5.015482,5.246168,5.476855
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,PALLD_F2_CA,Test,1912.398261,1287.035445,182415,302625,168183,84084,43535,23757,...,3.726484,3.931751,4.137019,4.342287,4.547555,4.752822,4.958090,5.163358,5.368626,5.573894
241,EZR_C3_NO,Test,1397.882540,862.498094,93846,244476,157603,87519,46906,24798,...,3.552562,3.768084,3.983606,4.199128,4.414650,4.630172,4.845695,5.061217,5.276739,5.492261
242,DNMBP_A3_NO,Test,1228.049113,686.319233,63539,216743,144264,64926,25444,10095,...,3.147823,3.403781,3.659739,3.915697,4.171655,4.427613,4.683571,4.939529,5.195487,5.451445
243,NUMB_E3_CA,Test,1104.526785,716.463072,261034,243954,85645,29137,10731,4287,...,3.614475,3.818583,4.022690,4.226798,4.430906,4.635014,4.839121,5.043229,5.247337,5.451445


In [40]:
df_8.to_csv('../results/histograms_std_entropy_8_bit.csv', index=False)

In [41]:
df_16.to_csv('../results/histograms_std_entropy_16_bit.csv', index=False)